In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import Tensor

In this set of exercises, we will consider recorded hits for muon positions along a single dimension. The recorded positions suffer from finite resolution and so carry an uncertainty. The recorded positions (hits) are then used to compute a variable per muon. You will need to compute the uncertainty on this value due to the finite resolution.

## Set up
First we want to set up our resolution. This should be a single-element tensor, called `resolution`, with a value of 100.0, that requires gradient:

In [3]:
resolution = torch.tensor([1e2], requires_grad=True)

In [4]:
assert resolution == 1e2
assert resolution.requires_grad

Now we want to take the true positions of the muons, `true_x`, and perturb them with the resolution. We assume that the resolution error contributes as a Gaussian:

In [5]:
true_x = torch.rand(100,1)

In [6]:
reco_x = true_x+(torch.randn_like(true_x)/resolution)
reco_x[:10]

tensor([[0.0805],
        [0.6654],
        [0.0671],
        [0.6379],
        [0.3920],
        [0.2181],
        [0.5894],
        [0.9845],
        [0.7067],
        [0.7014]], grad_fn=<SliceBackward0>)

Below is the function that will compute the variable from the the recorded hits:

In [7]:
def infer_theta(reco_hits:Tensor) -> Tensor:
    dz = 1.5
    theta = torch.arctan(reco_hits.abs()/dz)
    return theta

In [8]:
theta = infer_theta(reco_x)

In [9]:
theta[:10]

tensor([[0.0536],
        [0.4175],
        [0.0447],
        [0.4021],
        [0.2556],
        [0.1444],
        [0.3744],
        [0.5808],
        [0.4403],
        [0.4374]], grad_fn=<SliceBackward0>)

## Uncertainty
The uncertainty propagation formula for one dependent variable ($x$) is:
$$\sigma_y = \frac{\partial y}{\partial x}\sigma_x$$

Below, implement this formula and compute the uncertainty on the inferred theta angle for every muon.

In [10]:
from torch._vmap_internals import _vmap as vmap

def jacobian(y: Tensor, x: Tensor, create_graph: bool = False, allow_unused: bool = True) -> Tensor:
    r"""
    Computes the Jacobian (dy/dx) of y with respect to variables x. x and y can have multiple elements.
    If y has multiple elements then computation is vectorised via vmap.

    Arguments:
        y: tensor to be differentiated
        x: dependent variables
        create_graph: If True, graph of the derivative will
            be constructed, allowing to compute higher order derivative products.
            Default: False.
        allow_unused: If False, specifying inputs that were not
            used when computing outputs (and therefore their grad is always

    Returns:
        dy/dx tensor of shape y.shape+x.shape
    """

    if len(y) == 0:
        return None
    flat_y = y.reshape(-1)

    def get_vjp(v: Tensor) -> Tensor:
        return torch.autograd.grad(flat_y, x, v, retain_graph=True, create_graph=create_graph, allow_unused=allow_unused)[0].reshape(x.shape)

    return vmap(get_vjp)(torch.eye(len(flat_y), device=y.device)).reshape(y.shape + x.shape)

In [11]:
theta.shape, reco_x.shape

(torch.Size([100, 1]), torch.Size([100, 1]))

In [12]:
unc_theta = (jacobian(theta, reco_x)/resolution).sum((-1,-2))

In [13]:
unc_theta[:10]

tensor([[0.0066],
        [0.0056],
        [0.0067],
        [0.0056],
        [0.0062],
        [0.0065],
        [0.0058],
        [0.0047],
        [0.0055],
        [0.0055]], grad_fn=<SliceBackward0>)